In [1]:
%cd /content/drive/MyDrive/Poeran_Ortho_Price/

/content/drive/MyDrive/Poeran_Ortho_Price


In [2]:
import pandas as pd
import numpy as np

ecri_file_list = [ 'MtSinaiCustomData_2017.txt',
 'MtSinaiCustomData_2013.txt',   'MtSinaiCustomData_2018.txt',
 'MtSinaiCustomData_2014.txt', 'MtSinaiCustomData_2019.txt', 'MtSinaiCustomData_2015.txt',
 'MtSinaiCustomData_2016.txt']

ecri_df_list = []
for file in ecri_file_list:
  df = pd.read_csv(file, sep = "|").assign(Year = int(file.split('.')[0].split('_')[1]))
  ecri_df_list.append(df)


In [3]:

ecri_df = pd.concat(ecri_df_list, axis=0, ignore_index=True)
ecri_df.columns = ecri_df.columns.str.replace(" - ", " ").str.replace(" ","_") # Simplify all column names with an underscore

In [4]:
ecri_df.shape[0]

4918099

In [5]:
print('Number of Hospitals: {}'.format(ecri_df.groupby('Facility_Member_ID').size().to_frame()\
.reset_index().shape[0]) )

Number of Hospitals: 1689


In [6]:
print('Number of Implant Manufacturers: {}'.format(ecri_df.groupby('Sourcebase_Mfg_ID').size().to_frame()\
.reset_index().shape[0]) )

Number of Implant Manufacturers: 97


In [7]:
# Sum of total quantity and mean spend grouped by hospital id

ecri_tot_qty_mean_spend_corr_plot_df = pd.concat([
ecri_df.groupby('Facility_Member_ID')['Sum_Of_Qty'].sum().to_frame().reset_index().rename(columns = {'Sum_Of_Qty' : 'Per_ID_Sum_Of_Qty'}),

ecri_df.groupby('Facility_Member_ID')['Total_Spend'].mean().to_frame().reset_index().rename(columns = {'Total_Spend' : 'Per_ID_Mean_Of_Spend'})['Per_ID_Mean_Of_Spend'] ], axis = 1)

# **Most U.S Hospitals are Susceptible to a High Markup Potential in the Arthoplasty Surgical Implant Market**
> ###  **Condensing millions of transactions to capture high markup activity in a single plot**

The economics of arthoplasty surgical implant purchases by U.S. hospitals in relation to the high markups imposed by implant manufacturers remains an understudied research area. The arthosplasty implant market is nortoriously opaque with limited pricing transparacy, making it difficult for hospitals to know if they're getting a fair deal. Furthermore, surgical implants often have high markups -- sometimes 300-600% over manufacturing costs. This is due to large medical device companies maintaining [20-30% profit margins](https://www.healthcarevaluehub.org/advocate-resources/publications/medical-devices-worrying-parallels-our-nations-prescription-drug-concerns), with some products commanding even higher margins. The pricing of implants is often based on what the market and reimbursement system will [tolerate](https://starfishmedical.com/resource/margin-matters/), rather than production costs.

Here, I wanted to explore the landscape of the arthoplasty implant market from the perspective of a U.S. hospital. This data-driven analysis of hospital purchasing data from the [Economic Cycle Research Institute (ECRI)](https://www.businesscycle.com/) reveals alarming markup patterns in implant component transactions.





### **Data Summary**
*  Transactions between 2013 - 2019

*  Totaling over 4.5 million transctions

*  1,689 U.S. hospitals

*  97 implant manufacturers

*   Total U.S. hospital spend: \$10,513,927,584.14
*   Total U.S. hospital volume: 8,082,254










In [8]:
hospital_qty_tot_spend_avg_df = ecri_tot_qty_mean_spend_corr_plot_df.merge(ecri_df.groupby('Facility_Member_ID').size().to_frame()\
.rename(columns = {0 : 'Transaction_Count'}), on = 'Facility_Member_ID', how = 'left')\
.assign(Per_ID_Mean_Of_Spend = lambda s: round(s['Per_ID_Mean_Of_Spend'].astype(int), 2))\
.rename(columns = {'Per_ID_Sum_Of_Qty': 'Quantity total',  'Per_ID_Mean_Of_Spend': 'Spend average',
                                                                'Facility_Member_ID' : 'Facility id','Transaction_Count': 'Transaction count'})

In [ ]:
# hospital_qty_tot_spend_avg_df\
# .to_csv('hospital_qty_tot_spend_avg_df.csv', index=False)

### **The Intution Behind Visualzing Trends For Markup Potential**

While examining the relationship between average spend and total quantity might seem like the obvious approach to visualize implant purchasing trends, this analysis can yield counterintuitive results. Moreover, the lack of correlation between these variables reveals complex patterns than initially expected, necessitating a more sophisticated analytical approach.

For example, if hospital one purchased 98,584 implants and their average spend was \$2,218 and hospital two purchased 73,728 implants with an average spend of \$2,291, one could conclude hospital one is less susceptible to a high markup potential than hospital two due to its lower average spend and higher total quantity over a period of time.

In [ ]:
# !pip install -U kaleido

In [21]:
import plotly.express as px

fig = px.scatter(hospital_qty_tot_spend_avg_df,
                 x="Quantity total", y="Spend average", size = "Transaction count",
                 hover_data={'Quantity total': ':,', 'Spend average': ':$,.0f',
                             'Transaction count': ':,', 'Facility id': True},
                 title="Per-Hospital Relationship Between Spend Average and Quantity Total of Implants")
fig.update_layout(xaxis_title_text='Quantity total',
    yaxis_title_text='Spend average', title_font_size=15,
    autosize=False, width=685, height=415)

fig.show()

# fig.write_image('hospital_qty_tot_spend_avg.svg', format='svg')

fig.write_html("interactive_plots/hospital_qty_tot_spend_avg.html", full_html=False, include_plotlyjs='cdn')

In [10]:
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.897
Date:                Mon, 03 Mar 2025   Prob (F-statistic):             0.0889
Time:                        16:40:11   Log-Likelihood:                -14547.
No. Observations:                1689   AIC:                         2.910e+04
Df Residuals:                    1687   BIC:                         2.911e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2069.2124     36.806     56.219      0.000    1997.022    2141.403
x1             0.0062      0.004      1.702      0.089      -0.001       0.013
==============================================================================
Omnibus:                     2369.645   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           996943.295
Skew:                           7.761   Prob(JB):                         0.00
Kurtosis:                     121.005   Cond. No.                     1.15e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.15e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

 Here, I decided to visualize the two variables (average spend vs. quantity total) with each point representing a hospital. The size of the points correspond to the number of implant transactions over the course of 7 years.

After employing OLS regression analysis to test for correlation between average spend and quantity total, we found no significant relationship between these variables. (R<sup>2</sup>=0.002, p=0.089).

In [11]:
# Percentages of total spend and quantity grouped by hospital id and pecentage difference labels

ecri_qty_spend_corr_plot_df = pd.concat([pd.DataFrame( 100* (ecri_df.groupby('Facility_Member_ID')['Sum_Of_Qty'].sum() /
                               ecri_df['Sum_Of_Qty'].sum())   ).reset_index().rename(columns = {'Sum_Of_Qty' : 'Per_ID_Percentage_Of_Qty'}),
ecri_df.groupby('Facility_Member_ID')['Sum_Of_Qty'].sum().to_frame()\
.reset_index().rename(columns = {'Sum_Of_Qty' : 'Per_ID_Sum_Of_Qty'})['Per_ID_Sum_Of_Qty'] ,
pd.DataFrame( 100* (ecri_df.groupby('Facility_Member_ID')['Total_Spend'].sum() /
                               ecri_df['Total_Spend'].sum())   ).reset_index()\
                               .rename(columns = {'Total_Spend' : 'Per_ID_Percentage_Of_Total_Spend'})['Per_ID_Percentage_Of_Total_Spend'],
           ecri_df.groupby('Facility_Member_ID')['Total_Spend'].sum().to_frame()\
.reset_index().rename(columns = {'Total_Spend' : 'Per_ID_Total_Spend'})['Per_ID_Total_Spend'] ], axis = 1)\
.assign(Per_ID_Qty_Spend_Percent_Difference = lambda s: s['Per_ID_Percentage_Of_Qty'] - s['Per_ID_Percentage_Of_Total_Spend'],
        Per_ID_Qty_Spend_Percent_Difference_Threshold = lambda s: list( map(lambda x: "Less<br>Susceptible" if x > 0 else "More<br>Susceptible" ,s['Per_ID_Qty_Spend_Percent_Difference']))) # Percentage difference b/t quantity and spend



In [13]:
str(round( 100* ecri_qty_spend_corr_plot_df[ecri_qty_spend_corr_plot_df['Per_ID_Qty_Spend_Percent_Difference_Threshold'] == "More<br>Susceptible"].shape[0] /
          ecri_qty_spend_corr_plot_df.shape[0], 1)) + '% of hospitals are below the implant market threshold at zero'

'51.7% of hospitals are below the implant market threshold at zero'

### **Characterizing the arthoplasty implant market in a data-driven way**

 What if we took a different approach and compared **ratios or percentages** of total spend and quantity?


Going back to our  example scenario, if hospital one spent a total \$117 million for 98,584 implants and hospital two spent $68 million for 73,728 implants -- this will reveal that hospital one is more susceptible to a higher markup potential with a smaller percentage difference/change between quantity and spend of implants (hospital #1: 1.22% - 1.11% = **+0.11%**; hospital #2:  0.91% - 0.65% = **+0.26%**).

>> ##### **Note: The percentage change between implant quantity and spend is over a 7 year period, which factors in 8,082,254 of total U.S. hospital implant volume and $10,513,927,584.14 of total U.S. hospital spend, respectively.**

In [14]:
ecri_qty_spend_plot_df = ecri_qty_spend_corr_plot_df.merge(pd.concat([ecri_df.groupby('Facility_Member_ID').size().to_frame()\
.rename(columns = {0 : 'Transaction_Count'}),
ecri_df.groupby(['Facility_Member_ID'])['Sourcebase_Mfg_ID'].apply(set).to_frame()\
.assign(Manufacturer_Count = lambda s: list( map(lambda x: len(x) , s['Sourcebase_Mfg_ID']))),
ecri_df.groupby(['Facility_Member_ID'])['Standardized_Item_Description_Long'].apply(set).to_frame()\
.assign(Implant_Component_Count = lambda s: list( map(lambda x: len(x) , s['Standardized_Item_Description_Long'])))], axis =1), on = 'Facility_Member_ID', how = 'left')\
.rename(columns = {'Per_ID_Percentage_Of_Qty': 'Quantity percentage',  'Per_ID_Percentage_Of_Total_Spend': 'Spend percentage',
                   'Per_ID_Total_Spend': 'Spend total', 'Per_ID_Sum_Of_Qty': 'Quantity total',  'Facility_Member_ID' : 'Facility id',
                   'Transaction_Count': 'Transaction count', 'Per_ID_Qty_Spend_Percent_Difference_Threshold': 'Implant market behavior',
                   'Manufacturer_Count': 'Manufacturer count', 'Implant_Component_Count': 'Implant component type count'})

In [ ]:
# ecri_qty_spend_plot_df\
# .to_csv('hospital_qty_spend_pct_df.csv', index=False)

In [19]:

import plotly.express as px

fig = px.scatter(ecri_qty_spend_plot_df,
                 x="Quantity percentage", y="Spend percentage", size = "Transaction count", color = 'Implant market behavior',
                hover_data={'Implant market behavior': True,
                            'Quantity percentage':':.3f',
                            'Spend percentage': ':.3f',
                            'Manufacturer count': ':,',
                             'Implant component type count': ':,',
                            'Quantity total': ':,',
                            'Spend total': ':$,.0f',
                            'Transaction count': ':,',
                             'Facility id': True},
                 title="Per-Hospital Relationship Between Total Spend and Quantity of Implants")
fig.update_layout(xaxis_title_text='Quantity (%)', # xaxis label
    yaxis_title_text='Spend (%)',
    autosize=False, width=685, height=415, legend_title="")

fig.show()

fig.write_html("interactive_plots/hospital_qty_spend_pct.html", full_html=False, include_plotlyjs='cdn')

In [16]:
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 3.751e+04
Date:                Mon, 03 Mar 2025   Prob (F-statistic):               0.00
Time:                        16:43:37   Log-Likelihood:                 4112.3
No. Observations:                1689   AIC:                            -8221.
Df Residuals:                    1687   BIC:                            -8210.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0054      0.001      9.254      0.000       0.004       0.007
x1             0.9084      0.005    193.686      0.000       0.899       0.918
==============================================================================
Omnibus:                     1116.439   Durbin-Watson:                   1.855
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            56222.368
Skew:                           2.452   Prob(JB):                         0.00
Kurtosis:                      30.836   Cond. No.                         9.12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

OLS regression analysis suggests that using percentages and the changes assoicated with implant spend and quanity can, in fact, capture a significantly strong relationship across U.S. hospitals (R<sup>2</sup>=0.957, p<0.001).







The above plot recapitulates the hospital market landscape for implants with interpretable patterns and outliers. By characterizing the implant market in a data-driven way, we can reveal that most U.S. hospitals (51.7%) are more susceptible to a high markup potential during negotiations.  


Furthermore, this data-driven approach can also shed light on which implant manufacturers are **less likely** to deploy **high markups** in negotiations with hospitals across implant types.

In [ ]:
# Create a new column for implant type
ecri_df = ecri_df\
.assign(Implant_Type = lambda s: list( map(lambda x: 'Hip' if 'Hip' in x  else ( 'Knee' if 'Knee' in x else 'Other') ,s['UMDNS_Name'])),
        Implant_All_Types = lambda s: list( map(lambda x: 'Hip' if 'Hip' in x  else ( 'Knee' if 'Knee' in x
                                                                                     else ('Elbow' if 'Elbow' in x
                                                                                           else ('Ankle' if 'Ankle' in x
                                                                                                 else ( 'Shoulder' if 'Shoulder' in x
                                                                                                       else   ('Hand' if 'Hand' in x
                                                                                                               else ('Wrist' if 'Wrist' in x
                                                                                                                     else ( 'Finger/Thumb' if 'Finger/Thumb' in x
                                                                                                                           else ('Toe' if 'Toe' in x
                                                                                                                                 else   ('Foot' if 'Foot' in x
                                                                                                                                         else ('TMJ Fossa' if 'Fossa' in x
                                                                                                                                               else ('TMJ Condyle' if 'Condyle' in x
                                                                                                                                                     else 'TMJ'))))) ))  )  )   )) , s['UMDNS_Name'] )) )




In [ ]:
# Percentages of total spend and quantity grouped by manufacturer and implant type pairs

ecri_vendor_qty_spend_difference_plot_df = pd.concat([pd.DataFrame( 100* (ecri_df.groupby(['Sourcebase_Mfg_ID', 'Implant_Type'])['Sum_Of_Qty'].sum() /
                               ecri_df['Sum_Of_Qty'].sum())   ).reset_index().rename(columns = {'Sum_Of_Qty' : 'Per_ID_Implant_Percentage_Of_Qty'}),
ecri_df.groupby(['Sourcebase_Mfg_ID', 'Implant_Type'])['Sum_Of_Qty'].sum().to_frame()\
.reset_index().rename(columns = {'Sum_Of_Qty' : 'Per_ID_Implant_Sum_Of_Qty'})['Per_ID_Implant_Sum_Of_Qty'] ,
pd.DataFrame( 100* (ecri_df.groupby(['Sourcebase_Mfg_ID', 'Implant_Type'])['Total_Spend'].sum() /
                               ecri_df['Total_Spend'].sum())   ).reset_index()\
                               .rename(columns = {'Total_Spend' : 'Per_ID_Implant_Percentage_Of_Total_Spend'})['Per_ID_Implant_Percentage_Of_Total_Spend'],
           ecri_df.groupby(['Sourcebase_Mfg_ID', 'Implant_Type'])['Total_Spend'].sum().to_frame()\
.reset_index().rename(columns = {'Total_Spend' : 'Per_ID_Implant_Total_Spend'})['Per_ID_Implant_Total_Spend'] ], axis = 1)\
.merge( ecri_df[['Standardized_Mfg_Name_Long', 	'Sourcebase_Mfg_ID', 'Implant_Type']]\
       .drop_duplicates(['Sourcebase_Mfg_ID', 'Implant_Type']), on = ['Sourcebase_Mfg_ID', 'Implant_Type'], how ='left')\
.assign(Per_ID_Implant_Qty_Spend_Percent_Difference = lambda s: s['Per_ID_Implant_Percentage_Of_Qty'] - s['Per_ID_Implant_Percentage_Of_Total_Spend'],
        Per_ID_Implant_Qty_Spend_Percent_Difference_Threshold = lambda s: list( map(lambda x: 'Above' if x > 0 else 'Below' ,s['Per_ID_Implant_Qty_Spend_Percent_Difference']))) # Percentage difference b/t quantity and spend

### **The arthoplasty implant market is fragmented**

In the plot below, I stratified manufacturers into implant types (e.g. hip, knee, etc.) as a function of the percentage change between implant quantity and spend. This allows us to get a 7 year snapshot of the implant market  for more than 1,600 hospitals across 97 manufacturers.


To understand this chart in detail, scroll left to right hover over points for additional information in the tooltips. For example, hover over the knee implant type (the red point) of *Stryker Othopaedics Div Stryker Corp* --  where a majority of U.S. hospitals (n=1,144) were **less susceptible to a high markup potential** (among manufacturer and implant pairs=+1.41%) across more than 1,300 different knee implant components amounting to \$1.3 billion in total spend (tot. quantity=1.1 million), whereas well over 900 of U.S. hospitals were  **more susceptible to a high markup potential** (among manufacturer and implant pairs=-0.68%) during negotiations with *Smith and Nephew Inc Div Smith and Nephew plc* across nearly 1,600 different knee implant components amounting to $551 million in total spend over a 7 year timeframe
 (tot. quantity=369k).

This finding suggests fragmentation in the arthroplasty implant market, where manufacturers with more specialized components may create information asymmetry during negotiations. Consequently, this complexity makes it difficult for hospitals to stay current on all available options, thereby increasing their susceptibility to higher markups.

In [ ]:
plot_df = ecri_vendor_qty_spend_difference_plot_df\
                 .merge(pd.concat([ecri_df.groupby(['Sourcebase_Mfg_ID', 'Implant_Type']).size().to_frame()\
.rename(columns = {0 : 'Transaction_Count'}) , ecri_df.groupby(['Sourcebase_Mfg_ID', 'Implant_Type'])['Facility_Member_ID'].apply(set).to_frame()\
.assign(Hospital_Count = lambda s: list( map(lambda x: len(x) , s['Facility_Member_ID']))),
ecri_df.groupby(['Sourcebase_Mfg_ID', 'Implant_Type'])['Standardized_Item_Description_Long'].apply(set).to_frame()\
.assign(Implant_Component_Count = lambda s: list( map(lambda x: len(x) , s['Standardized_Item_Description_Long']))) ], axis =1)    ,
on = ['Sourcebase_Mfg_ID', 'Implant_Type'], how = 'left')\
.rename({'Transaction_Count': 'Transaction count', 'Per_ID_Implant_Qty_Spend_Percent_Difference': 'Percentage difference',
         'Standardized_Mfg_Name_Long': 'Maufacturer name',
         'Implant_Type':'Implant type', 'Hospital_Count': 'Hospital count', 'Implant_Component_Count': 'Implant component type count',
         'Per_ID_Implant_Sum_Of_Qty': 'Implant component total quantity', 'Per_ID_Implant_Total_Spend': 'Implant component total spend' }, axis = 1)

In [ ]:
plot_df\
.to_csv('mfg_mkt_landscape_by_implant_df.csv', index = False)

In [ ]:
!pwd

In [ ]:
import plotly.express as px


fig = px.scatter(plot_df ,
         x="Maufacturer name",  y="Percentage difference", color="Implant type",  hover_data={'Percentage difference':':.6f',
                                                                                              'Implant component total spend':':$,.2f',
                                                                                              'Implant component total quantity': ':,',
                                                                                              'Implant component type count': ':,',
                                                                                              'Transaction count': ':,',
                                                                                              'Hospital count': ':,'},
         title="Manufacturer Market Landscape by Implant Type")

fig.update_layout(bargap=0.1,
    xaxis_title_text='Manufacturer',
    yaxis_title_text='% Change (Quantity - Spend)<br>  &#8592; More Sus   Less Sus &#8594;', legend_title="",
                  autosize=False, width=3000, height=680)
fig.update_traces(marker={'size': 8})
fig.update_xaxes(tickangle=-45)   #,  categoryorder= 'total descending'
fig.show()

# fig.write_image('mfg_mkt_landscape_by_implant.svg', format='svg')
fig.write_html("interactive_plots/mfg_mkt_landscape_by_implant.html", full_html=False, include_plotlyjs='cdn')



**TODO:**  

*   Visualize the top and bottom-20 hip and knee implants on the quantity-spend differential
*   Ask about hospital management survey
*   Look for other publicly available data (e.g. gudid data?) that is revelant and could be promising for correlates







In [ ]:
ecri_df_18 = ecri_df\
.assign(Load_Date = lambda s:  pd.to_datetime(s['Load_Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') )\
.query("Load_Date >= '2014-03-01' and Load_Date <= '2019-12-31'").sort_values('Load_Date')


ecri_vendor_qty_spend_difference_line_plot_df = pd.concat([pd.DataFrame( 100* (ecri_df_18.groupby(['Load_Date',  'Implant_Type'])['Sum_Of_Qty'].sum() /
                               ecri_df_18['Sum_Of_Qty'].sum())   ).reset_index().rename(columns = {'Sum_Of_Qty' : 'Per_ID_Implant_Percentage_Of_Qty'}),
ecri_df_18.groupby(['Load_Date', 'Implant_Type'])['Sum_Of_Qty'].sum().to_frame()\
.reset_index().rename(columns = {'Sum_Of_Qty' : 'Per_ID_Implant_Sum_Of_Qty'})['Per_ID_Implant_Sum_Of_Qty'] ,
pd.DataFrame( 100* (ecri_df_18.groupby(['Load_Date', 'Implant_Type'])['Total_Spend'].sum() /
                               ecri_df_18['Total_Spend'].sum())   ).reset_index()\
                               .rename(columns = {'Total_Spend' : 'Per_ID_Implant_Percentage_Of_Total_Spend'})['Per_ID_Implant_Percentage_Of_Total_Spend'],
           ecri_df_18.groupby(['Load_Date', 'Implant_Type'])['Total_Spend'].sum().to_frame()\
.reset_index().rename(columns = {'Total_Spend' : 'Per_ID_Implant_Total_Spend'})['Per_ID_Implant_Total_Spend'] ], axis = 1)\
.merge( ecri_df_18[['Load_Date',	 'Implant_Type']]\
       .drop_duplicates([ 'Implant_Type']), on = ['Load_Date', 'Implant_Type'], how ='left')\
.assign(Per_ID_Implant_Qty_Spend_Percent_Difference = lambda s: s['Per_ID_Implant_Percentage_Of_Qty'] - s['Per_ID_Implant_Percentage_Of_Total_Spend'],
        Per_ID_Implant_Qty_Spend_Percent_Difference_Threshold = lambda s: list( map(lambda x: 'Less Susceptible' if x > 0 else 'More Susceptible' ,s['Per_ID_Implant_Qty_Spend_Percent_Difference']))) # Percentage difference b/t quantity and spend

In [ ]:
ecri_vendor_qty_spend_difference_line_plot_df\
.to_csv('hospital_qty_spend_difference_temporal_line_plot_df.csv', index=False)


### **Temporal relationships of susceptibility behavior across implants**

So far, I've visualized years of condensed data showing how hospitals exhibit susceptibility behaviors during negotiations with implant manufacturers. Now the question emerges: can we meaningfully interpret these temporal patterns (Apr 2014 - Dec 2019) of hospital susceptibility behavior across different implant categories?

Visualizing the temporal dynamics of implant market activity reveals compelling patterns. For instance, consider the economic conditions during knee and hip implant negotiations in early 2019.

Data from ECRI's U.S. coincident index suggests the country was experiencing an inflation downswing during this period, potentially enabling hospitals to secure more favorable pricing for hip and knee implants.


>> ##### **Note: "Load Date" represents the date ECRI loaded implant market data into their database and does't reflect the actual date of the transaction. The day of each transction was not provided in this dataset (only the month, year). Using "Load Date" to visualize temporal dynamics was sufficient since there  was a month or two difference from the actual transaction.**   

In [ ]:
annual_mkt_activity_df = pd.read_csv('data/hospital_qty_spend_difference_temporal_line_plot_df.csv')\
.set_index(['Load_Date', 'Implant_Type'])['Per_ID_Implant_Qty_Spend_Percent_Difference'].unstack().reset_index()\
.sort_values('Load_Date')


fig = px.line(annual_mkt_activity_df, x="Load_Date", y=annual_mkt_activity_df.columns,
              hover_data={"Load_Date": "|%B %d, %Y" },
              title='Annual Market Activity by Implant Type', category_orders={"Implant_Type": ["Knee", "Hip", "Other"]})
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(yaxis_title_text='% Change (Quantity - Spend) <br>  &#8592; More Sus   Less Sus &#8594;', xaxis_title_text='Load Date',   legend_title="",
                  autosize=False, width=3500)


fig.show()
# fig.write_image('annual_mkt_activity_by_implant.svg', format='svg')



fig.write_html("interactive_plots/annual_mkt_activity_by_implant.html", full_html=False, include_plotlyjs='cdn')